### URL HRRR Automatic Download

Notebook contents 
* Another attempt a downloading HRRR data 

created by Cassie Lumbrazo\
last updated: Jan 2025\
run location: UAS linux\
python environment: **herbie**\
*created a new environment for herbie data downloader...*

In [1]:
# import packages 
%matplotlib inline

# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

sns.set_theme()
# plt.rcParams['figure.figsize'] = [12,6] #overriding size

# data packages 
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime, timedelta

import scipy

In [5]:
# import urllib.request
import os
import requests
import pygrib
import xarray as xr
import numpy as np
from datetime import datetime, timedelta


In [7]:
# Define parameters
BASE_URL = "https://pando-rgw01.chpc.utah.edu/hrrr/sfc"  # HRRR Alaska Pando archive
SAVE_DIR = '/home/cassie/data/hrrr_ak_download/hrrr_ak_netcdf_test'   # Directory to save data
# SAVE_DIR = "./HRRR_AK_NetCDF"  # Directory to save data
START_DATE = "2024-01-01"  # Start date (YYYY-MM-DD)
END_DATE = "2024-01-02"  # End date (YYYY-MM-DD)

In [8]:
# Create save directory if not exists
os.makedirs(SAVE_DIR, exist_ok=True)

# Function to download a GRIB2 file
def download_grib(date, hour, forecast_hour, save_path):
    file_name = f"hrrr.t{hour:02d}z.wrfsfcf{forecast_hour:02d}.ak.grib2"
    file_url = f"{BASE_URL}/{date}/{file_name}"
    try:
        response = requests.get(file_url, stream=True)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=1024):
                    f.write(chunk)
            print(f"Downloaded: {file_name}")
            return True
        else:
            print(f"Failed to download {file_name}: HTTP {response.status_code}")
            return False
    except Exception as e:
        print(f"Error downloading {file_name}: {e}")
        return False

# Function to process GRIB2 to NetCDF
def grib_to_netcdf(grib_path, netcdf_path):
    try:
        grbs = pygrib.open(grib_path)
        data = []
        lats, lons = None, None

        # Extract data for each message
        for grb in grbs:
            if lats is None or lons is None:
                lats, lons = grb.latlons()
            data.append(grb.values)

        grbs.close()

        # Convert to NetCDF using xarray
        data_array = xr.DataArray(
            np.stack(data),
            dims=["time", "lat", "lon"],
            coords={
                "time": [datetime.strptime(grib_path[-25:-9], "hrrr.t%Hz.wrfsfcf")],
                "lat": lats[:, 0],
                "lon": lons[0, :]
            }
        )
        data_array.to_netcdf(netcdf_path)
        print(f"Saved NetCDF: {netcdf_path}")
    except Exception as e:
        print(f"Error processing {grib_path}: {e}")

# Main function
def download_hrrr_to_netcdf(start_date, end_date):
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    while current_date <= end_date:
        date_str = current_date.strftime("%Y%m%d")
        for hour in range(24):  # Loop over all model run hours
            for forecast_hour in range(19):  # HRRR-AK provides forecasts up to 18 hours
                grib_path = os.path.join(SAVE_DIR, f"hrrr.t{hour:02d}z.wrfsfcf{forecast_hour:02d}.ak.grib2")
                netcdf_path = grib_path.replace(".grib2", ".nc")

                # Skip if NetCDF already exists
                if os.path.exists(netcdf_path):
                    continue

                # Download and process
                if download_grib(date_str, hour, forecast_hour, grib_path):
                    grib_to_netcdf(grib_path, netcdf_path)
                    os.remove(grib_path)  # Clean up GRIB2 file to save space
        current_date += timedelta(days=1)

# Run the script
if __name__ == "__main__":
    download_hrrr_to_netcdf(START_DATE, END_DATE)


Failed to download hrrr.t00z.wrfsfcf00.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf01.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf02.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf03.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf04.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf05.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf06.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf07.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf08.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf09.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf10.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf11.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf12.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf13.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf14.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf15.ak.grib2: HTTP 404
Failed to download hrrr.t00z.wrfsfcf16.ak.grib2: HTTP 404
Failed to down

KeyboardInterrupt: 

In [ ]:
# # Define the start and end dates
# start_date = datetime(2023, 10, 1)
# end_date = datetime(2023, 10, 2)

# # Base URL for HRRR Alaska data on Pando archive
# base_url = 'https://pando-rgw01.chpc.utah.edu/hrrr/sfc/'

# # Loop through each day in the date range
# current_date = start_date
# while current_date <= end_date:
#     date_str = current_date.strftime('%Y%m%d')
#     for hour in range(24):  # Model runs every hour
#         run_hour = f'{hour:02d}'
#         for forecast_hour in range(19):  # Forecast hours from 00 to 18
#             forecast_str = f'{forecast_hour:02d}'
#             file_name = f'hrrr.t{run_hour}z.wrfsfcf{forecast_str}.ak.grib2'
#             file_url = f'{base_url}{date_str}/{file_name}'
#             save_path = f'/path/to/save/directory/{file_name}'
#             try:
#                 urllib.request.urlretrieve(file_url, save_path)
#                 print(f'Successfully downloaded {file_name}')
#             except Exception as e:
#                 print(f'Failed to download {file_name}: {e}')
#     current_date += timedelta(days=1)